In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9881831915401129, 0.9864172135286532)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9993173375851955, 0.9992738522583102)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,NaN,NaN,Error,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,NaN,String,EMail,PhoneNumber,Error,NaN,Boolean,NumericId
1,NaN,NaN,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,NaN,NaN,PhoneNumber,NaN,NaN,NaN,NumericId
2,NaN,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,Name,Surname,String,NaN,PhoneNumber,NaN,BiologicalSex,Boolean,Error
3,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,Name,Surname,String,EMail,PhoneNumber,Date,NaN,NaN,NumericId
4,NaN,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,CountryCode,Name,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
6,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId
7,Error,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,NaN,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,NaN,NumericId
8,ItalianFiscalCode,NaN,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,NaN,Name,NaN,NaN,EMail,PhoneNumber,Date,NaN,Boolean,NumericId
9,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Boolean,NaN


In [10]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Country', 'Error'): 285,
         ('NaN', 'CountryCode'): 89,
         ('Error', 'ItalianFiscalCode'): 24,
         ('Error', 'CountryCode'): 17,
         ('CountryCode', 'NaN'): 71,
         ('String', 'Municipality'): 108,
         ('String', 'Error'): 58,
         ('Error', 'Document'): 163,
         ('Country', 'String'): 234,
         ('Error', 'String'): 35,
         ('Country', 'Name'): 362,
         ('Error', 'Name'): 39,
         ('Error', 'Country'): 144,
         ('Error', 'BiologicalSex'): 26,
         ('NaN', 'Error'): 14,
         ('Integer', 'Error'): 41,
         ('Country', 'Document'): 295,
         ('Boolean', 'BiologicalSex'): 21,
         ('Boolean', 'Country'): 9,
         ('String', 'Address'): 1585,
         ('ProvinceCode', 'Boolean'): 14,
         ('Country', 'Municipality'): 209,
         ('Address', 'String'): 1058,
         ('Municipality', 'Country'): 22,
         ('Name', 'Boolean'): 17,
         ('Name', 'Country'): 397,
         ('Surname', '